In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
#porterstemmer is used for stemming
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
import re
import pandas as pd
import chardet as chardet
#Data cleaning & preprocessing
import nltk
nltk.download('stopwords')
#Deep Learning (Embedding & LSTM)
from tensorflow.keras.preprocessing.text import one_hot
from numpy import array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense,LSTM
from tensorflow import keras
import tensorflow as tf
from tensorflow import convert_to_tensor, string
from tensorflow.keras.layers import TextVectorization, Embedding, Layer
from tensorflow.data import Dataset
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Reading CSV

In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/Datasets/stockmarket/Combined_News_DJIA.csv')
df.shape

In [ ]:
df.Date.min(),df.Date.max()

('2008-08-08', '2016-07-01')

In [ ]:
df.isnull().sum()

Date     0
Label    0
Top1     0
Top2     0
Top3     0
Top4     0
Top5     0
Top6     0
Top7     0
Top8     0
Top9     0
Top10    0
Top11    0
Top12    0
Top13    0
Top14    0
Top15    0
Top16    0
Top17    0
Top18    0
Top19    0
Top20    0
Top21    0
Top22    0
Top23    1
Top24    3
Top25    3
dtype: int64

In [ ]:
df.dropna(how='any',axis=0,inplace=True)
df.reset_index(inplace=True)
df.shape

(1986, 28)

In [ ]:
df['Label'].value_counts()

1    1062
0     924
Name: Label, dtype: int64

Splitting into Train, Test and doing Preprocessing

In [ ]:
train=df[df["Date"]<'2015-01-01']
test=df[df["Date"]>='2015-01-01']

text=train.iloc[:,3:].columns
#Text cleanning and Preprocessing for Train data

for col1 in text:
  for row1 in range (0,len(train)):
    review=re.sub('[^a-zA-Z]',' ',train[col1][row1])
    review=review.lower()
    review=review.split()
    #review=[ps.stem(word) for word in review]
    review=' '.join(review)
    train[col1][row1]=(review)
train["News"]=train[text].agg('  '.join, axis=1)
X_train=train[['Date','News']]
Y_train=train["Label"]
#Text cleanning and Preprocessing For Test data
for col2 in text:
  for row2 in range (1608,test.index.max()):
    review=re.sub('[^a-zA-Z]',' ',test[col2][row2])
    review=review.lower()
    review=review.split()
    #review=[ps.stem(word) for word in review]
    review=' '.join(review)
    test[col2][row2]=(review)
test["News"]=test[text].agg('  '.join, axis=1)
X_test=test[['Date','News']]
Y_test=test["Label"]

<ipython-input-10-35a15c00515e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col1][row1]=(review)
<ipython-input-10-35a15c00515e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["News"]=train[text].agg('  '.join, axis=1)
<ipython-input-10-35a15c00515e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[col2][row2]=(review)
<ipython-input-10-35a15c00515e>:27

In [ ]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((1608, 2), (1608,), (378, 2), (378,))

In [ ]:
#unique words count from a column of a dataframe
from collections import Counter
results = Counter()
X_train["News"].str.lower().str.split().apply(results.update)
print(len(results))

30439


Converting text to vector using Embedding layers,but Embedding layers only takes numerical values hence first we need to convert the text to encoded vectors
2 ways to do that
1. One_hot  
2. Text Vectorization  
---

1 Converting text to vector(Using one_hot encoding)for feeding into Embedding layer






In [ ]:
# vocab_size=2000
max_words_list=2500
# embed_dim=64
encod_corp=[]
for i,doc in enumerate(X_train["News"]):
    encod_corp.append(one_hot(doc,2000))
padded_corp = pad_sequences(encod_corp,maxlen=max_words_list,padding='post')



---
2: Converting text to vector(Using Text Vectorization)for feeding into Embedding Layer of LSTM


In [ ]:
sentences=tuple(train["News"])
output_sequence_length = 300
vocab_size = 2000
# Create the TextVectorization layer
vectorize_layer = TextVectorization(
                  output_sequence_length=output_sequence_length,
                  max_tokens=vocab_size)
# Train the layer to create a dictionary
vectorize_layer.adapt(sentences)
# Convert all sentences to tensors
word_tensors = convert_to_tensor(sentences, dtype=tf.string)
# Use the word tensors to getvectorized  phrases
vectorized_words = vectorize_layer(word_tensors)
print("Vocabulary: ", vectorize_layer.get_vocabulary())
print("Vectorized words: ", vectorized_words)


#Output:
'''Vocabulary:  ['', '[UNK]', 'the','targeting', 'slams', 'sharia', 'review', 'raised', 'putting', 'pot', 'planes', 'morning', 'monsanto', 'loss', 'legalization', 'iranians', 'holy', 'holds', 'hidden', 'guy', 'gangs', 'eyes', 'commercial', 'ceo', 'attempted', 'atlantic', 'abbott', 'visa', 'swine', 'surprise', 'sharing', 'sexually', 'sets', 'scientific', 'ring', 'reserves', 'prosecutors', 'programme', 'prepares', 'possibly', 'panel', 'ones', 'lot', 'lord', 'lama', 'km']
Vectorized words:  tf.Tensor(
[[   7  780    1 ...    2  275    4]
 [   7  201    1 ...  249    0    0]
 [   7 1609   13 ...  780    8  295]
 ...
 [ 776  170    1 ...  422    1   47]
 [  36    1  493 ... 1836 1021 1089]
 [   1  973  103 ...  367    1  252]], shape=(1608, 300), dtype=int64)'''


# output_length = 300
# word_embedding_layer = Embedding(vocab_size, output_length)
# embedded_words = word_embedding_layer(vectorized_words)
# print(embedded_words)

Using Deep neural network RNN LSTM


In [ ]:
embed_dim = 24
lstm_out = 16
batch_size = 5

model = Sequential()
model.add(Embedding(50, embed_dim,input_length = padded_corp.shape[1]))
model.add(LSTM(lstm_out,activation='relu',kernel_regularizer=keras.regularizers.l2()))
model.add(Dense(1,activation='sigmoid'))
optimizers = keras.optimizers.Adam(lr=0.001)
model.compile(loss = 'binary_crossentropy', optimizer=optimizers,metrics = ['accuracy'])
print(model.summary())
model.fit(padded_corp,Y_train,epochs=5,verbose=1)

In [ ]:
#Accuracy
scores = model.evaluate(padded_corp_1, Y_test,verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

If we are using ML Models:
> Convert text data to Vectors using:

1. Bag of words
2. TFIDF
3. Word2Vec
---

1)Bag of words

In [ ]:
# #Creating a bag of words model
# from sklearn.feature_extraction.text import CountVectorizer
# cv=CountVectorizer(binary=True,ngram_range=(2,2))
# X_train_bow=cv.fit_transform(new_df["News"]).toarray()
# X_train_bow.shape

---
2)TF-IDF



In [ ]:
# #Creating a tfidf model
# from sklearn.feature_extraction.text import TfidfVectorizer
# #max features =2500 ------> it means that out of total max occuring words only top 2500 words will be selected for training the model
# tf=TfidfVectorizer(ngram_range=(2,2))
# X=tf.fit_transform(new_df["News"]).toarray()
#Y_train_bow=new_df['Label']
#X_train_bow.shape,Y_train_bow.shape

ML Models:


---
Naive_bayes


In [ ]:
from sklearn.naive_bayes import MultinomialNB
stock_model=MultinomialNB().fit(X,Y)

In [ ]:
y_pred=stock_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
score=accuracy_score(Y_test,y_pred)
score

0.4973544973544973

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,Y_test))

              precision    recall  f1-score   support

           0       0.11      0.45      0.17        44
           1       0.88      0.50      0.64       334

    accuracy                           0.50       378
   macro avg       0.49      0.48      0.41       378
weighted avg       0.79      0.50      0.58       378



RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
classifier.fit(X,Y)

In [ ]:
y_pred_rdf=classifier.predict(X_test)

In [ ]:
score=accuracy_score(Y_test,y_pred_rdf)
print(score)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred_rdf,Y_test))

              precision    recall  f1-score   support

           0       0.02      0.40      0.04        10
           1       0.97      0.51      0.66       368

    accuracy                           0.50       378
   macro avg       0.50      0.45      0.35       378
weighted avg       0.94      0.50      0.65       378



All In One ML Models

In [ ]:
def get_models():
  models = dict()
  #models['XGB'] = XGBClassifier()
  #models['LGBM'] = LGBMClassifier()
  models['lr'] = LogisticRegression()
  models['knn'] = KNeighborsClassifier()
  models['cart']= DecisionTreeClassifier()
  models['rnf'] = RandomForestClassifier()
  models['svm'] = SVC(probability=True)
  models['bayes'] = GaussianNB()
  models['SGD'] = SGDClassifier()

  return models

def model_predict(x,y,model,xt):
  model.fit(x,y)
  Y_pre = model.predict(xt)
  # Y_pred_prob = model.predict_proba(X_test_1)
  return Y_pre

def model_pre_pro(x,y,model,xt):
  model.fit(x,y)
  Y_pred_prob = model.predict_proba(xt)
  return Y_pre

models = get_models()


Score = []
Algo =[]
for name, model in models.items():
    Y_pre = model_predict(X_train_1,Y_train,model,X_test_1)
    Score_DT = accuracy_score(Y_test,Y_pre)
    #F1 = f1_score(Y_test,Y_pre)
    #print('>%s % .3f'% (name,Score_DT))
    Algo.append(name)
    Score.append(Score_DT )

Classification_result = pd.DataFrame(list(zip(Algo,Score)), columns=['Alo_name','Accuracy'])
Classification_result
